- -> exp037
- fold の分割を改善

In [1]:
%load_ext autoreload

In [2]:
import os

is_kaggle_notebook = os.path.exists("/kaggle/input")

# 必要パッケージをインストール
if is_kaggle_notebook:
    !pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
    !pip install /kaggle/input/torch-geometric-2-6-1/torch_geometric-2.6.1-py3-none-any.whl

In [3]:
import os
import sys
import warnings
from pathlib import Path

import lightgbm as lgb
import numpy as np
import pandas as pd
import torch
import wandb
from rdkit import rdBase
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GroupKFold, KFold
from tqdm.auto import tqdm

rdBase.DisableLog("rdApp.warning")
warnings.filterwarnings("ignore")

In [23]:
pr_number = 1
debug = True
augmented_feature = '["morgan", "maccs"]'
graph_pooling = "sum"
notes = ""
is_trimmer_cyclic = False
exp = "exp047"
batch_size = 256
num_epochs = 1000
drop_ratio = 0.5
set_seed = True
ignore_error_asterisk = False

In [24]:
if set_seed:
    SEED = 42
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    g = torch.Generator()
    g.manual_seed(SEED)

    # # 決定論モード
    # torch.use_deterministic_algorithms(True)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True
    

In [25]:
%autoreload 2

if is_kaggle_notebook:
    module_path = f"/kaggle/input/myproject-pr-{pr_number:04}"
    !mkdir src
    !cp -r $module_path/* src/
    src_path = "./"
else:
    src_path = "../"

sys.path.append(src_path)

from src.data import add_descriptors, add_external_data, load_data, make_smile_canonical, generate_trimer
from src.model import get_model
from src.utils import NULL_FOR_SUBMISSION, generate_scaffold, score, add_scaffold_kfold, scaffold_cv_split
from src.utils.upload_kaggle_dataset import (
    create_kaggle_dataset_metadata,
    upload_kaggle_dataset,
)

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [26]:
model_names = [
    "gnn",
    # "bfgnn", "dir", "grin", "irm", "lstm", "rpgnn", "smiles_transformer", "ssr"
]
config = {
    "debug": debug,
    "n_splits": 3,
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "drop_ratio": drop_ratio,
    "graph_pooling": graph_pooling,
    "is_trimmer_cyclic": is_trimmer_cyclic,
    "ignore_error_asterisk": ignore_error_asterisk,
}
smiles_col = "SMILES_trimmer_cyclic" if is_trimmer_cyclic else "SMILES"

data_exp = "exp047"

if is_kaggle_notebook:
    config["debug"] = False

if config["debug"]:
    config["n_splits"] = 2
    config["num_epochs"] = 10

targets = ["Tg", "FFV", "Tc", "Density", "Rg"]
org_target_cols = [f"org_{target}" for target in targets]        

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if isinstance(augmented_feature, str):
    augmented_feature = eval(augmented_feature)

cuda


In [28]:
# ---------------------------
# メイン処理
# ---------------------------
if config["debug"]:
    output_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / exp / "debug"
    data_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / exp / "debug"
else:
    output_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / exp

model_output_path = output_path / "model_cv"
if not os.path.exists(model_output_path):
    os.makedirs(model_output_path)

if is_kaggle_notebook:
    # kaggle notebook
    data_dir = Path("/kaggle/input")
else:
    # local
    data_dir = Path("/home/kouya-takahashi/kaggle/opp2025/data/raw")

if config["debug"]:
    train_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / data_exp / "train_debug.csv"
else:
    train_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / data_exp / "train.csv"

    
if os.path.exists(train_path):
    print("Load train.csv")
    
    train = pd.read_csv(train_path)
else:
    train, _ = load_data(data_dir)
    # 外部データ判定用に元の目的変数を保持しておく
    # 外部データ判定は scaffold_cv_split 内で行う
    for target in targets:
        train[f"org_{target}"] = train[target]
    

    train["org_SMILES"] = train["SMILES"]
    train["SMILES"] = train["SMILES"].apply(make_smile_canonical)
    
    if config["debug"]:
        # 各ターゲットが欠損していないデータを30 件ずつ取り出す
        tmp_dfs = []
        for target in targets:
            cond = train[target].notnull()
            tmp_dfs.append(train[cond].iloc[:100])
        train = pd.concat(tmp_dfs).reset_index(drop=True)
    else:
        print(train.shape)
        external_data_dict = [
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv",
                "col": "Tg",
            },
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv",
                "col": "Tc",
                "rename_d": {"TC_mean": "Tc"},
            },
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv",
                "col": "FFV",
            },
            {
                "ex_path": data_dir / "tg-smiles-pid-polymer-class/TgSS_enriched_cleaned.csv",
                "col": "Tg",
            },
            {
                "ex_path": data_dir / "smiles-extra-data/data_dnst1.xlsx",
                "col": "Density",
                "rename_d": {"density(g/cm3)": "Density"}, 
            },
            {
                "ex_path": data_dir / "smiles-extra-data/data_tg3.xlsx",
                "col": "Tg",
                "rename_d": {"Tg [K]": "Tg"}, 
            },
            {
                "ex_path": data_dir / "smiles-extra-data/JCIM_sup_bigsmiles.csv",
                "col": "Tg",
                "rename_d": {"Tg (C)": "Tg"}, 
            },
        ]
        for d in external_data_dict:
            print(f"ex_path: {str(d['ex_path'])}")
            train = add_external_data(
                df=train,
                ex_path=d.get("ex_path"),
                col=d.get("col"),
                rename_d=d.get("rename_d"),
                is_complement=True
            )
            print("after train.shape: ", train.shape)

    train["is_external"] = train["id"].isnull()
    
    # トリマー環状化合物を生成
    if config["is_trimmer_cyclic"]:
        error_cnt = 0
        trimer_smiles = []
        for smiles in tqdm(train["SMILES"].values):
            try:
                trimer_smiles.append(generate_trimer(smiles))
            except ValueError as e:
                print(f"smiles: {smiles}, {e}")
                trimer_smiles.append(smiles)
                error_cnt += 1
        train["SMILES_trimmer_cyclic"] = trimer_smiles
        print(f"error smiles count: {error_cnt}")
    
    train["id"] = np.arange(len(train))
    train = train.reset_index(drop=True)
    print("Save train.csv")
    train.to_csv(train_path, index=False)


# 計算済の fold を突合
folds = pd.read_csv("/home/kouya-takahashi/kaggle/opp2025/data/preprocess/fold/folds.csv")
train = train.merge(folds[["SMILES", "fold"]], how="left", on="SMILES")
print(train.shape)

if not config["ignore_error_asterisk"]:
    # 何もしない
    cond = [True for _ in range(len(train))]
elif not config["is_trimmer_cyclic"]:
    # アスタリスクの数が 2つ以外は除外
    cond = train["SMILES"].str.count("\*") == 2
else:
    # トリマー環状化に成功してたら、SMILES_trimmer_cyclic にアスタリスクは含まれない
    cond = train["SMILES_trimmer_cyclic"].str.count("\*") == 0

train = train.loc[cond].reset_index(drop=True)
print(train.shape)

submission = pd.read_csv(
    data_dir / "neurips-open-polymer-prediction-2025/sample_submission.csv"
)

fold_results = []

all_loss_tables = []
for model_name in model_names:
    all_models = {}
    
    wandb_name = f"{exp}_{model_name}" if not config["debug"] else f"{exp}_{model_name}_debug"
    wandb.init(project="opp2025", name=f"{exp}_{model_name}", config=config)
    wandb.log({"Notes": notes})
    loss_table = wandb.Table(columns=["exp", "model_name", "fold", "target", "mae", "mse"])
    oof_dfs = []
    mae_dict = {}
    all_models = {}
    
    print("====================")
    print(f"model_name: {model_name}")

    for target in targets:
        models = []
        cond_target = train[target].notnull()
        if not config["ignore_error_asterisk"]:
            # 何もしない
            cond = [True for _ in range(len(train))]
        elif not config["is_trimmer_cyclic"]:
            # アスタリスクの数が 2つ以外は除外
            cond = train["SMILES"].str.count("\*") == 2
        else:
            # トリマー環状化に成功してたら、SMILES_trimmer_cyclic にアスタリスクは含まれない
            cond = train["SMILES_trimmer_cyclic"].str.count("\*") == 0
        
        df_train = train[cond_target & cond].reset_index(drop=True)
        X = df_train[[smiles_col]]
        y = df_train[target]

        oof = np.full(len(X), np.nan, dtype=float)

        for fold, train_idx, val_idx in scaffold_cv_split(df_train, target=target, n_splits=config["n_splits"]):
            if len(train_idx) == 0 or len(val_idx) == 0:
                print(f"Skip fold... train_idx: {len(train_idx)}, val_idx: {len(val_idx)}")
                continue
            print(f'\n=== Fold {fold + 1} / {config["n_splits"]} ===')

            X_train = X.iloc[train_idx][smiles_col].to_list()
            y_train = y.iloc[train_idx].to_numpy()
            X_val = X.iloc[val_idx][smiles_col].to_list()
            y_val = y.iloc[val_idx].to_numpy()
    
            model = get_model(model_name)(
                task_type="regression",
                num_task=1,
                batch_size=config["batch_size"],
                augmented_feature=augmented_feature,
                epochs=config["num_epochs"],
                drop_ratio=config["drop_ratio"],
                patience=50,
                scheduler_patience=5,
                verbose=False,
                graph_pooling=config["graph_pooling"]
            )
            model.fit(
                # gnn.autofit(
                X_train=X_train,
                y_train=y_train,
                X_val=X_val,
                y_val=y_val,
                # n_trials=1,
            )
            save_model_path = model_output_path / f"{model_name}_{fold}_{target}.pt"
            model.save(path=str(save_model_path))
            models.append(model)
    
            preds = model.predict(X_val)["prediction"].flatten()
            oof[val_idx] = preds
            
            mse = mean_squared_error(y_val, preds)
            mae = mean_absolute_error(y_val, preds)
            print(f"Fold {fold+1} target: {target} MSE: {mse:.4f} | MAE: {mae:.4f}")
            loss_table.add_data(exp, model_name, fold, target, mae, mse)

            all_loss_tables.append({"target": target, "fold": fold, "mae": mae})
            
        # 評価に使うのは元データのみ
        cond = (~np.isnan(oof)) & (df_train[f"org_{target}"].notnull())
        y_true = y[cond]
        y_pred = oof[cond]
        score_mse = mean_squared_error(y_true, y_pred)
        score_mae = mean_absolute_error(y_true, y_pred)
        print(f"RMSE for {target}: {score_mse:.4f}")
        print(f"MAE for {target}: {score_mae:.4f}")
        mae_dict[target] = score_mae

        oof_df = pd.DataFrame({
            "id": df_train["id"],
            target: oof            
        })
        oof_dfs.append(oof_df)

        wandb.log({f"fold_target_losses": loss_table})

    
    # CV 計算
    cond = ~train["is_external"]
    oof_df = pd.DataFrame()
    
    # 元の目的変数も入れておく
    for target in targets:
        oof_df[f"org_{target}"] = train.loc[cond, f"org_{target}"]
    
    oof_df["id"] = train.loc[cond, "id"]
    
    # 正規化前の SMILES
    oof_df["SMILES"] = train.loc[cond, "org_SMILES"]
    
    for i_oof in oof_dfs:
        oof_df = oof_df.merge(i_oof, on="id", how="left")
    
    oof_df.to_csv(output_path / "oof.csv", index=False)
    
    solution = train.loc[cond, ["id"] + org_target_cols].copy()
    solution.columns = ["id"] + targets
    
    # solution = solution.fillna(NULL_FOR_SUBMISSION)
    # oof_df = oof_df.fillna(NULL_FOR_SUBMISSION)
    
    # 評価
    final_score = score(
        solution=solution,
        submission=oof_df,
    )
    print(f"\n📊 Final OOF Score (wMAE): {final_score:.6f}")
    wandb.log({"wMAE": final_score})

    
    # target 毎の 平均 mae を記録
    for target in targets:
        key_name = f"{target}_mean_mae"
        mae_values = mae_dict[target]
        # mae_values = [d["mae"] for d in all_loss_tables if d["target"] == target]
        wandb.log({key_name: np.mean(mae_values)})


Load train.csv
(500, 16)
(496, 16)


wandb: ERROR Control-C detected -- Run data was not synced


KeyboardInterrupt: 